##Load data from ADLS

In [2]:
CONTAINER_INPUT = '<YOUR_CONTAINER>'
FILENAME = "/mnt/{}/UsedCars.csv".format(CONTAINER_INPUT)

UsedCars = (spark
  .read
  .option("header", True)
  .option("inferSchema", True) \
  .option("sep", ',') \
  .csv(FILENAME)
)

In [3]:
display(UsedCars)

In [4]:
UsedCars.printSchema()

##Create temporary table from Spark dataframe

In [6]:
UsedCars.createOrReplaceTempView('UsedCars_Temp')

###Check if exists any missing data

In [8]:
%sql
select * 
from UsedCars_Temp 
where Price is null
or Age is null
or KM is null
or FuelType is null
or HP is null
or MetColor is null
or Automatic is null
or CC is null
or Doors is null
or Weight is null

###Fill missing values with zero

In [10]:
UsedCars = UsedCars.fillna(0)

In [11]:
UsedCars.createOrReplaceTempView('UsedCars_Temp')

In [12]:
%sql
select * 
from UsedCars_Temp 
where Price is null
or Age is null
or KM is null
or FuelType is null
or HP is null
or MetColor is null
or Automatic is null
or CC is null
or Doors is null
or Weight is null

In [13]:
display(UsedCars)

In [14]:
%sql
select * from UsedCars_Temp where Price = 0

###Delete rows with no target

In [16]:
from pyspark.sql import functions as f

display(UsedCars.filter(f.col('Price') == 0))

In [17]:
UsedCars = UsedCars.filter(f.col('Price') != 0)

##Persist DataFrame to parquet in ADLS

In [19]:
CONTAINER_OUTPUT = 'cleaneddata'
UsedCars.write.format('parquet').save('/mnt/{}/UsedCars'.format(CONTAINER_OUTPUT))